In [ ]:
import numpy as np
import pandas as pd
year = 1977
rd_actual = pd.read_excel('rdindex' + str(year) + '.xlsx')

if year == 1977:
    rd_actual = rd_actual.dropna(axis=0)
    rd_actual['RD no.'] = pd.to_numeric(rd_actual['RD no.'].astype(str).str.split('.').str[0], errors='coerce').astype(float).astype('Int64')
    rd_actual['rdid'] = rd_actual['RD no.'].astype(str).str[:3] + '0' + rd_actual['RD no.'].astype(str).str[3:]
    rd_actual['rdid'] = rd_actual['rdid'].astype(int)
else:
    rd_actual['rdid'] = rd_actual['RD no.'].astype(str).apply(lambda x: x[:3] + '0' + x[3:]).astype(int)

column_length = rd_actual['CPC code'].astype(str).str.len()
print("CPC code length", set(column_length))

rd_actual.tail(2)
# rd.rename(columns={'year': 'Year', 'Predicted_Label': 'cpc_class'}, inplace=True)

In [ ]:
years = [1977, 1976, 1973, 1974, 1972]
rd = pd.read_csv('rd_prediction_bert-base-uncased_32_batch_size_alpha_10.csv', usecols=['rdid', 'year', 'Predicted_Label'])
rd = rd[rd['year'].isin(years)]
rd.head(2)


In [ ]:
rd

In [ ]:
import numpy as np
import pandas as pd

# List of years
# years = [1977, 1976, 1973, 1974, 1972]  # Add more years as needed

# Create an empty list to store the DataFrames
dfs = []

# Read and process data for each year
for year in years:
    filename = 'rdindex' + str(year) + '.xlsx'
    
    if year == 1977:
        rd_actual = pd.read_excel(filename)
        rd_actual = rd_actual.dropna(axis=0)
        rd_actual['RD no.'] = pd.to_numeric(rd_actual['RD no.'].astype(str).str.split('.').str[0], errors='coerce').astype(float).astype('Int64')
        rd_actual['rdid'] = rd_actual['RD no.'].astype(str).str[:3] + '0' + rd_actual['RD no.'].astype(str).str[3:]
        rd_actual['rdid'] = rd_actual['rdid'].astype(int)
        # display(rd_actual['rdid'])
    elif year == 1976:
        rd_actual = pd.read_excel(filename)
        rd_actual['rdid'] = rd_actual['RD no.'].astype(str).apply(lambda x: x[:3] + '0' + x[3:]).astype(int)
        # display(rd_actual)
    
    elif((year == 1973) or (year == 1974)):
        filename = str(year) + '_RD - Sheet1.csv'
        rd_actual = pd.read_csv(filename)
        rd_actual['CPC code'] = rd_actual['CPC Class'].astype(str)
        rd_actual['rdid'] = rd_actual['RD No'].astype(str).apply(lambda x: x[:3] + '0' + x[3:]).astype(int)
        
    elif(year == 1972):
        filename = str(year) + '_RD - Sheet1.csv'
        rd_actual = pd.read_csv(filename)
        rd_actual['CPC code'] = rd_actual['CPC Class'].astype(str)
        rd_actual['rdid'] = rd_actual['RD No'].astype(str).apply(lambda x: x[:3] + x[3:]).astype(int)
        # display(rd_actual['rdid'])
        # display(rd_actual)
    dfs.append(rd_actual)

# Concatenate the DataFrames into a single DataFrame
rd_actual = pd.concat(dfs)

# Print CPC code length
column_length = rd_actual['CPC code'].astype(str).str.len()
print("CPC code length:", set(column_length))

rd_actual = rd_actual[['CPC code', 'rdid']]
rd_actual.reset_index(drop = True, inplace = True)
# Print the tail of the merged DataFrame
display(rd_actual)


In [ ]:
# #for 2 cpc code length
# # Merge the 'cpc' column from rd_actual to rd based on 'rdid'
# merged_df = rd.merge(rd_actual[['rdid', 'CPC code']], on='rdid', how='left')
# display(merged_df)
# # print(merged_df.year.value_counts())
# merged_df = merged_df.dropna(axis=0)
# merged_df = merged_df.reset_index(drop=True)
# merged_df['cpc_actual'] = merged_df['CPC code'].str[:-2]
# merged_df['Predicted_Label'] = merged_df['Predicted_Label'].str[:-1]
# merged_df_non_null = merged_df.dropna(subset=['cpc_actual'])
# # Remove the brackets ( ) around the text in 'cpc_actual' column
# merged_df_non_null['cpc_actual'] = merged_df_non_null['cpc_actual'].str.replace(r"\(|\)", "")
# merged_df_non_null = merged_df_non_null[merged_df_non_null['cpc_actual'].str.len() == 2]
# # merged_df_non_null
# # pd.set_option('display.max_rows', None)
# # merged_df_non_null[["rdid", "Predicted_Label", "cpc_actual", "year"]]

In [ ]:
#For 3 cpc code length
# Merge the 'cpc' column from rd_actual to rd based on 'rdid'
merged_df = rd.merge(rd_actual[['rdid', 'CPC code']], on='rdid', how='left')
display(merged_df)
# print(merged_df.year.value_counts())
merged_df = merged_df.dropna(axis=0)
merged_df = merged_df.reset_index(drop=True)
merged_df['cpc_actual'] = merged_df['CPC code'].str[:-1]
merged_df_non_null = merged_df.dropna(subset=['cpc_actual'])
# Remove the brackets ( ) around the text in 'cpc_actual' column
merged_df_non_null['cpc_actual'] = merged_df_non_null['cpc_actual'].str.replace(r"\(|\)", "")
merged_df_non_null = merged_df_non_null[merged_df_non_null['cpc_actual'].str.len() == 3]
# merged_df_non_null
# pd.set_option('display.max_rows', None)
# merged_df_non_null[["rdid", "Predicted_Label", "cpc_actual", "year"]]

In [ ]:
merged_df_non_null

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming you have the merged_df_non_null DataFrame

# Extract the predicted label and actual values columns
y_pred = merged_df_non_null['Predicted_Label']
y_actual = merged_df_non_null['cpc_actual']

# Calculate accuracy
accuracy = accuracy_score(y_actual, y_pred)

# Calculate precision
precision = precision_score(y_actual, y_pred, average='macro')

# Calculate recall
recall = recall_score(y_actual, y_pred, average='macro')

# Calculate F1-score
f1 = f1_score(y_actual, y_pred, average='macro')

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

# Assuming you have the merged_df_non_null DataFrame

# Extract the predicted label and actual values columns
y_pred = merged_df_non_null['Predicted_Label']
y_actual = merged_df_non_null['cpc_actual']

# Extract the first letter from the CPC classes
y_pred_first_letter = y_pred.apply(lambda x: x[0])
y_actual_first_letter = y_actual.apply(lambda x: x[0])

# Get the unique first letters from both labels
unique_classes = sorted(set(y_actual_first_letter) | set(y_pred_first_letter))

# Create a confusion matrix
cm = confusion_matrix(y_actual_first_letter, y_pred_first_letter, labels=unique_classes)

# Convert the confusion matrix to a DataFrame for better visualization
cm_df = pd.DataFrame(cm, index=unique_classes, columns=unique_classes)

# Sort the DataFrame by overall performance (you can choose any performance metric, e.g., accuracy, precision, recall, etc.)
# Here, we'll sort by accuracy (sum of true positives on the diagonal) in descending order
sum_by_class = cm_df.sum(axis=1)
sorted_classes = sum_by_class.sort_values(ascending=False).index
cm_df_sorted = cm_df.loc[sorted_classes, sorted_classes]

# Create a heatmap using seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(cm_df_sorted, annot=True, cmap='Blues', fmt='d')
plt.xlabel('Predicted CPC Sectors')
plt.ylabel('Actual CPC Sectors')
plt.title('Confusion Matrix on sectors (Ranked)')
plt.tight_layout()
# Save the figure in EPS format with width of 20 cm and no extra spaces
plt.savefig('confusion_matrix_ranked_1.eps', format='eps', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
import pandas as pd

def class_wise_accuracy(confusion_matrix_df):
    class_wise_accuracy_dict = {}
    for class_label in confusion_matrix_df.index:
        true_positives = confusion_matrix_df.loc[class_label, class_label]
        false_negatives = confusion_matrix_df.loc[class_label, :].sum() - true_positives
        class_accuracy = true_positives / (true_positives + false_negatives)
        class_wise_accuracy_dict[class_label] = class_accuracy

    return class_wise_accuracy_dict

# Assuming that the confusion matrix is stored in a pandas DataFrame called cm_df_sorted
# You can sort the DataFrame to make it easier to interpret the results
cm_df_sorted = cm_df_sorted.sort_index(axis=0).sort_index(axis=1)

# Calculate class-wise accuracy using the function
class_wise_acc = class_wise_accuracy(cm_df_sorted)

# Print the class-wise accuracy
print("Class-wise Accuracy:")
for class_label, accuracy in class_wise_acc.items():
    print(f"Sector {class_label}: {accuracy:.4f}")


In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

# Assuming you have the merged_df_non_null DataFrame

# Extract the predicted label and actual values columns
y_pred = merged_df_non_null['Predicted_Label']
y_actual = merged_df_non_null['cpc_actual']

# Extract the first letter from the CPC classes
y_pred_first_letter = y_pred.apply(lambda x: x[0:2])
y_actual_first_letter = y_actual.apply(lambda x: x[0:2])

# Get the unique first letters from both labels
unique_classes = sorted(set(y_actual_first_letter) | set(y_pred_first_letter))

# Create a confusion matrix
cm = confusion_matrix(y_actual_first_letter, y_pred_first_letter, labels=unique_classes)

# Convert the confusion matrix to a DataFrame for better visualization
cm_df = pd.DataFrame(cm, index=unique_classes, columns=unique_classes)

# Sort the DataFrame by overall performance (you can choose any performance metric, e.g., accuracy, precision, recall, etc.)
# Here, we'll sort by accuracy (sum of true positives on the diagonal) in descending order
sum_by_class = cm_df.sum(axis=1)
sorted_classes = sum_by_class.sort_values(ascending=False).index
cm_df_sorted = cm_df.loc[sorted_classes, sorted_classes]

# Create a heatmap using seaborn
plt.figure(figsize=(12, 8))
sns.heatmap(cm_df_sorted, annot=True, cmap='Blues', fmt='d')
plt.xlabel('Predicted CPC Class (First Two Digits)')
plt.ylabel('Actual CPC Class (First Two Digits)')
plt.title('Confusion Matrix on sectors (Ranked)')
plt.tight_layout()
# Save the figure in EPS format with width of 20 cm and no extra spaces
plt.savefig('confusion_matrix_ranked_2.eps', format='eps', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
print(cm_df)

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming you have the merged_df_non_null DataFrame

# Extract the predicted label and actual values columns
y_pred = merged_df_non_null['Predicted_Label'].str[0]
y_actual = merged_df_non_null['cpc_actual'].str[0]

# Calculate accuracy
accuracy = accuracy_score(y_actual, y_pred)

# Calculate precision
precision = precision_score(y_actual, y_pred, average='macro')

# Calculate recall
recall = recall_score(y_actual, y_pred, average='macro')

# Calculate F1-score
f1 = f1_score(y_actual, y_pred, average='macro')

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
